In [1]:
#Webservice que faz a busca da chave da Lider
    
from suds.client import Client

url = "http://mundoincrivel.com.br/fonetica.asmx?wsdl"

cliente = Client(url)

resultado = cliente.service.Transformar('THEREZA')
print resultado

TRSA 


In [2]:
#Importando bibliotecas necessarias
import nltk
from nltk.stem.snowball import SnowballStemmer
import pandas  as pd
from soundex import Soundex
from Fonetica import fonetica #Classe importada para melhora a legibilidade do código

In [3]:
#Funçoes utilitarias de testes

def busca_nome(palavra):     
    chaveFonetica = chave.metaphoneBR(palavra)
    resultadoMPBR = nomes_com_chave[nomes_com_chave['METAPHONEBR']==chaveFonetica.strip()].NOME
    
    chaveFonetica =  cliente.service.Transformar(palavra)
    resultadoLider = nomes_com_chave[nomes_com_chave['LIDER']==chaveFonetica.strip()].NOME  
    
    chaveFonetica = stemmerRSLP.stem(palavra)
    resultadoRSLP = nomes_com_chave[nomes_com_chave['RSLP']==chaveFonetica.upper()].NOME  
    
    chaveFonetica = chavesoundex.soundex(palavra)
    resultadoSOUND = nomes_com_chave[nomes_com_chave['CHAVESOUNDEX']==chaveFonetica].NOME  
    
    chaveFonetica = chave.chavefoneticaBR(unicode(palavra),False)
    resultadoBSBRP = nomes_com_chave[nomes_com_chave['BUSCABR']==chaveFonetica].NOME  
    
    chaveFonetica = chave.chavefoneticaBR(unicode(palavra),True)
    resultadoBSBRV = nomes_com_chave[nomes_com_chave['BUSCABRV']==chaveFonetica].NOME  
    
    chaveFonetica = stemmersnow.stem(palavra)
    resultadoSNOW = nomes_com_chave[nomes_com_chave['SNOW']==chaveFonetica.upper()].NOME      

    data = {"RSLP":{"NOMESENCONTRADOS":list(resultadoRSLP.iloc[:].values)},\
            "SNOW":{"NOMESENCONTRADOS":list(resultadoSNOW.iloc[:].values)},\
            "CHAVESOUNDEX":{"NOMESENCONTRADOS":list(resultadoSOUND.iloc[:].values)},\
            "BUSCABRV":{"NOMESENCONTRADOS":list(resultadoBSBRV.iloc[:].values)},\
            "BUSCABR":{"NOMESENCONTRADOS":list(resultadoBSBRP.iloc[:].values)},\
            "METAPHONEBR":{"NOMESENCONTRADOS":list(resultadoMPBR.iloc[:].values)},\
            "LIDER":{"NOMESENCONTRADOS":list(resultadoLider.iloc[:].values)}
            } 
    return data

def contar_ocorrencias(esperado,nomesEncontrados):
    acertos = 0
    fp = 0
    fn = 0
    fpl= 0
    fnl = 0
    for index,item in enumerate(esperado):
        if item in nomesEncontrados:
            acertos += 1
        else:
            fn += 1
            fnl += contar_ocorrencias_levenshtein(item,nomesEncontrados,verifica_fn)
    #calcular falsos positivos
    for index,item in enumerate(nomesEncontrados):
        if item not in esperado:
            fp += 1
            fpl += contar_ocorrencias_levenshtein(item,esperado,verifica_fp)
    #print('Esperado: ', esperado , ' Encontrados: ' , nomesEncontrados , ' Acertos: ' , str(acertos), 'FP',fp,'FN',fn )     
    return acertos,fp,fn,fpl,fnl

def contar_ocorrencias_levenshtein(busca,esperados,func):
    count = 0
    for index,item in enumerate(esperados):
        count += func(nltk.edit_distance(item, busca)) 
    return count
def verifica_fp(distance):
    fpl = 0 
    #Considerar somente os erros com 2 pontos de distancia
    if distance > 2:
        fpl = 1
    return fpl
def verifica_fn(distance):
    fnl = 0 
    #Considerar somente os erros com 2 pontos de distancia
    if distance <= 2:
        fnl = 1       
    return fnl

In [4]:
#Inicializacao dos objetos 
chavesoundex = Soundex()
stemmerRSLP = nltk.stem.RSLPStemmer()
stemmersnow = SnowballStemmer("portuguese")
chave = fonetica()

#Carga do arquivo que tem os nomes e as respectivas chaves geradas anteriormente ja gravadas, simulando uma base de dados
nomes_com_chave = pd.read_csv('nomes_com_chaves_ws.csv')
#Carga do arquivo que contem os nomes para busca e os resultados esperados para cada nome de acordo com a base utilizada
buscasdf = pd.read_csv(filepath_or_buffer='buscas_milton.csv', sep=';',encoding='utf-8')


In [5]:
records = []


sumacertos = [0,0,0,0,0,0,0]
sumfp = [0,0,0,0,0,0,0]
sumfn = [0,0,0,0,0,0,0]
sumfpl = [0,0,0,0,0,0,0]
sumfnl = [0,0,0,0,0,0,0]

for index,row in buscasdf.iterrows():
    busca = row['BUSCA']
    data = busca_nome(busca)
    resultado = row['Retorno Esperado'].split(',')
    
    #Dados de cada algoritmo
    acertos = [0,0,0,0,0,0,0]
    fp = [0,0,0,0,0,0,0]
    fn = [0,0,0,0,0,0,0]
    fpl = [0,0,0,0,0,0,0]
    fnl = [0,0,0,0,0,0,0]    
    #RSLP
    nomesEncontrados = data['RSLP']['NOMESENCONTRADOS']
    acertos[0],fp[0],fn[0],fpl[0],fnl[0] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[0] += acertos[0]
    sumfp[0] += fp[0]
    sumfn[0] += fn[0]
    sumfpl[0] += fpl[0]
    sumfnl[0] += fnl[0]
    
    
    #SNOWBALL
    nomesEncontrados = data['SNOW']['NOMESENCONTRADOS']
    acertos[1],fp[1],fn[1],fpl[1],fnl[1] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[1] += acertos[1]
    sumfp[1] += fp[1]
    sumfn[1] += fn[1]
    sumfpl[1] += fpl[1]
    sumfnl[1] += fnl[1]
    
    #SOUNDEX
    nomesEncontrados = data['CHAVESOUNDEX']['NOMESENCONTRADOS']
    acertos[2],fp[2],fn[2],fpl[2],fnl[2] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[2] += acertos[2]
    sumfp[2] += fp[2]
    sumfn[2] += fn[2]
    sumfpl[2] += fpl[2]
    sumfnl[2] += fnl[2]
    
    #BUSCABRV
    nomesEncontrados = data['BUSCABRV']['NOMESENCONTRADOS']
    acertos[3],fp[3],fn[3],fpl[3],fnl[3] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[3] += acertos[3]
    sumfp[3] += fp[3]
    sumfn[3] += fn[3]
    sumfpl[3] += fpl[3]
    sumfnl[3] += fnl[3]
    
    
    #BUSCABR
    nomesEncontrados = data['BUSCABR']['NOMESENCONTRADOS']
    acertos[4],fp[4],fn[4],fpl[4],fnl[4] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[4] += acertos[4]
    sumfp[4] += fp[4]
    sumfn[4] += fn[4]
    sumfpl[4] += fpl[4]
    sumfnl[4] += fnl[4]
    
    #METAPHONE
    nomesEncontrados = data['METAPHONEBR']['NOMESENCONTRADOS']
    acertos[5],fp[5],fn[5],fpl[5],fnl[5] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[5] += acertos[5]
    sumfp[5] += fp[5]
    sumfn[5] += fn[5]
    sumfpl[5] += fpl[5]
    sumfnl[5] += fnl[5]
    
    #LIDER
    nomesEncontrados = data['LIDER']['NOMESENCONTRADOS']
    acertos[6],fp[6],fn[6],fpl[6],fnl[6] = contar_ocorrencias(resultado,nomesEncontrados)
    sumacertos[6] += acertos[6]
    sumfp[6] += fp[6]
    sumfn[6] += fn[6]
    sumfpl[6] += fpl[6]
    sumfnl[6] += fnl[6]
    
    linha ={"BUSCA":busca,"Resultado Esperado": row['Retorno Esperado'],\
            'RSLP': data['RSLP']['NOMESENCONTRADOS'],"ACRSLP": str( acertos[0]),"FPRSLP": str(fp[0]),"FNRSLP":str(fn[0]),\
            'SNOW': data['SNOW']['NOMESENCONTRADOS'],"ACSNOW": str( acertos[1]),"FPSNOW": str(fp[1]),"FNSNOW":str(fn[1]),\
            'CHAVESOUNDEX': data['CHAVESOUNDEX']['NOMESENCONTRADOS'],"ACCHAVESOUNDEX": str( acertos[2]),"FPCHAVESOUNDEX": str(fp[2]),"FNCHAVESOUNDEX":str(fn[2]),\
            'BUSCABRV': data['BUSCABRV']['NOMESENCONTRADOS'],"ACBUSCABRV": str( acertos[3]),"FPBUSCABRV": str(fp[3]),"FNBUSCABRV":str(fn[3]),\
            'BUSCABR': data['BUSCABR']['NOMESENCONTRADOS'],"ACBUSCABR": str( acertos[4]),"FPBUSCABR": str(fp[4]),"FNBUSCABR":str(fn[4]),\
            'METAPHONEBR': data['METAPHONEBR']['NOMESENCONTRADOS'],"ACMETAPHONEBR": str(acertos[5]),"FPMETAPHONEBR": str(fp[5]),"FNMETAPHONEBR":str(fn[5]),\
            'LIDER': data['LIDER']['NOMESENCONTRADOS'],"ACLIDER": str( acertos[6]),"FPLIDER": str(fp[6]),"FNLIDER":str(fn[6]) }
    
    records.append(linha)
    
    
#Gerar linha com sumarizacao    
linha ={"BUSCA":"","Resultado Esperado": "",\
        'RSLP': "","ACRSLP": str( sumacertos[0]),"FPRSLP": str(sumfp[0]),"FNRSLP":str(sumfn[0]),\
        'SNOW': "","ACSNOW": str( sumacertos[1]),"FPSNOW": str(sumfp[1]),"FNSNOW":str(sumfn[1]),\
        'CHAVESOUNDEX': "","ACCHAVESOUNDEX": str( sumacertos[2]),"FPCHAVESOUNDEX": str(sumfp[2]),"FNCHAVESOUNDEX":str(sumfn[2]),\
        'BUSCABRV':"","ACBUSCABRV": str( sumacertos[3]),"FPBUSCABRV": str(sumfp[3]),"FNBUSCABRV":str(sumfn[3]),\
        'BUSCABR': "","ACBUSCABR": str( sumacertos[4]),"FPBUSCABR": str(sumfp[4]),"FNBUSCABR":str(sumfn[4]),\
        'METAPHONEBR': "","ACMETAPHONEBR": str( sumacertos[5]),"FPMETAPHONEBR": str(sumfp[5]),"FNMETAPHONEBR":str(sumfn[5]),\
        'LIDER': "","ACLIDER": str( sumacertos[6]),"FPLIDER": str(sumfp[6]),"FNLIDER":str(sumfn[6]) }

records.append(linha)
 
    
#Gerar metrica F1

# Os nomes das colunas devem seguir o mesmo padrao por causa da serialização em arquivo do pandas
linha = {"BUSCA":"Algoritmo","Resultado Esperado":"Precision","RSLP":"Recall","ACRSLP":"F1"}
records.append(linha)

algoritmos =["RSLP","SNOWBALL","SOUNDEX","BUSCABRV","BUSCABR","METAPHONEBR","LIDER"]

for x in range(0, 7): 
    print('Gerando calculos algoritmo: ' + algoritmos[x])
    
    precision = 0.0
    recall = 0
    f1 = 0.0
    
    precisionl = 0.0
    recalll = 0.0
    f1l = 0.0
    
    if (sumacertos[x]+sumfp[x]) > 0 and (sumacertos[x]+sumfn[x]) > 0:
        precision = float(sumacertos[x])/(sumacertos[x]+sumfp[x])
        recall = float(sumacertos[x])/(sumacertos[x]+sumfn[x])
        f1 = float(2*(precision*recall)/(precision+recall))
        
    if (sumacertos[x]+sumfpl[x]) > 0 and (sumacertos[x]+sumfnl[x]) > 0:
        precisionl = float(sumacertos[x])/(sumacertos[x]+sumfpl[x])
        recalll = float(sumacertos[x])/(sumacertos[x]+sumfnl[x])
        f1l = float(2*(precisionl*recalll)/(precision+recalll))   
    
    linha = {"BUSCA":algoritmos[x],"Resultado Esperado":'{percent:.2%}'.format(percent=precision).replace('.',','),
             "RSLP":'{percent:.2%}'.format(percent=recall).replace('.',','),"ACRSLP":'{percent:.2%}'.format(percent=f1).replace('.',',')}
      
    records.append(linha)
    
    linha = {"BUSCA":algoritmos[x] + " Levenshtein","Resultado Esperado":'{percent:.2%}'.format(percent=precisionl).replace('.',','),
             "RSLP":'{percent:.2%}'.format(percent=recalll).replace('.',','),"ACRSLP":'{percent:.2%}'.format(percent=f1l).replace('.',',')}
      
    records.append(linha)    

df = pd.DataFrame(records,columns=["BUSCA","Resultado Esperado",\
                                   "RSLP","ACRSLP","FPRSLP","FNRSLP",\
                                   "SNOW","ACSNOW","FPSNOW","FNSNOW",
                                   "CHAVESOUNDEX","ACCHAVESOUNDEX","FPCHAVESOUNDEX","FNCHAVESOUNDEX",
                                   "BUSCABRV","ACBUSCABRV","FPBUSCABRV","FNBUSCABRV",
                                   "BUSCABR","ACBUSCABR","FPBUSCABR","FNBUSCABR",
                                   "METAPHONEBR","ACMETAPHONEBR","FPMETAPHONEBR","FNMETAPHONEBR",
                                   "LIDER","ACLIDER","FPLIDER","FNLIDER"])  



#Gravando o resultado em CSV
df.to_csv(path_or_buf='testes_automaticos_lev1.csv',index=False,encoding='utf-8',sep=";")    

C:\Users\Vinicius Martins\Anaconda2\lib\site-packages\pandas\core\ops.py:683: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  result = lib.scalar_compare(x, y, op)
C:\Users\Vinicius Martins\Anaconda2\lib\site-packages\ipykernel\__main__.py:36: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Vinicius Martins\Anaconda2\lib\site-packages\ipykernel\__main__.py:43: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Vinicius Martins\Anaconda2\lib\site-packages\nltk\metrics\distance.py:46: UnicodeWarning: Unicode unequal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  c = lev[i - 1][j - 1] + (c1 != c2)


Gerando calculos algoritmo: RSLP
Gerando calculos algoritmo: SNOWBALL
Gerando calculos algoritmo: SOUNDEX
Gerando calculos algoritmo: BUSCABRV
Gerando calculos algoritmo: BUSCABR
Gerando calculos algoritmo: METAPHONEBR
Gerando calculos algoritmo: LIDER
